# INSTALL LIBRARIES

In [5]:
# !pip3 install tensorflow_datasets
# !pip3 install pandas

# IMPORT LIBRARIES

In [6]:
# Warnings configuration
import warnings
warnings.filterwarnings('ignore')

# General Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

# Neural Network Components
from tensorflow import keras
from keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K

warnings.filterwarnings('always')

In [7]:
from ANN_creator_module import NeuralNetworkConstructor

# READING THE DATA SET

In [8]:
df_path = '/tf/Face_ID1/list_attr_celeba.txt'
df = pd.read_csv(df_path, skiprows=1 ,sep=r'\s+')

In [9]:
# df.head()

# CREATING THE TENSORFLOW DATASET

In [10]:
images_dir = '/tf/keras_neural_network/Mis_Tests/img_align_celeba/'
resize_factor = 2
H = 218 // resize_factor
W = 178 // resize_factor
img_size = 128  # usar H = W si quieres cuadrado
batch_size = 32
minitrain_size = int(202599*.70)
minival_size = int(202599*.30)

In [11]:
df.replace(-1, 0, inplace=True)

removed_columns = [
    'Sideburns','Wearing_Earrings','Mouth_Slightly_Open','Heavy_Makeup',
    'Attractive','Rosy_Cheeks','Wearing_Hat','Wearing_Lipstick',
    'Wearing_Necklace','Wearing_Necktie'
]
df.drop(removed_columns, axis=1, inplace=True)

In [12]:
df.shape

(202599, 30)

In [13]:
label_cols = df.columns
y = df[label_cols].values.astype(np.float32)
paths = (images_dir + df.index).values

In [14]:
idx = np.arange(len(paths))
np.random.seed(67)
np.random.shuffle(idx)

train_idx = idx[:minitrain_size]
val_idx = idx[minitrain_size : minitrain_size + minival_size]

train_paths = paths[train_idx]
train_labels = y[train_idx]

val_paths = paths[val_idx]
val_labels = y[val_idx]

In [15]:
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (img_size, img_size)) / 255.0
    return img, label

In [28]:
# ====== MINI TRAIN DATASET ======
train_ds = (
    tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
)

# ====== MINI VAL DATASET ======
val_ds = (
    tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size).prefetch(tf.data.AUTOTUNE)
)

# BUILDING THE MODEL

In [29]:
from tensorflow.keras.applications import DenseNet121

In [30]:
inputs = keras.Input((img_size, img_size, 3))
num_of_classes = 30

In [31]:
base_model = DenseNet121(
    include_top=False,
    input_shape=(img_size, img_size, 3),
    input_tensor=inputs, pooling=None)
base_model.trainable = True

In [32]:
x = inputs
x = base_model(x)
x = layers.BatchNormalization()(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)

x = layers.Dense(
    units=520, 
    activation='relu', 
    name='dense1', 
    kernel_regularizer = regularizers.L1L2(l1=1e-5, l2=1e-4))(x)

x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)

outputs = layers.Dense(
    units=num_of_classes, 
    activation='sigmoid', 
    name='clasification_layer', 
    kernel_regularizer = regularizers.L1L2(l1=1e-5, l2=1e-4))(x)

In [33]:
model = keras.Model(inputs,outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, 4, 4, 1024)        7037504   
                                                                 
 batch_normalization_2 (Bat  (None, 4, 4, 1024)        4096      
 chNormalization)                                                
                                                                 
 global_average_pooling2d_1  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense1 (Dense)              (None, 520)               5330

# TRAINING THE MODEL

In [34]:
model.compile(
    loss="binary_crossentropy", 
    optimizer=eval(f"keras.optimizers.{'RMSprop'}(learning_rate=0.01)"), 
    metrics=["binary_accuracy"]
)

In [35]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy',
    min_delta=0.001,
    patience=7, 
    restore_best_weights=True,
    mode="auto",
    verbose=1,
    baseline=None
)

In [36]:
ReduceLROnPlateau = keras.callbacks.ReduceLROnPlateau(
    monitor='val_binary_accuracy',
    factor=0.1,
    patience=2,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0.00001,
)

In [37]:
ModelCheckpoint = keras.callbacks.ModelCheckpoint(
    '/tf/Face_ID1/my_model2.keras',
    monitor='val_binary_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
    initial_value_threshold=None,
)

In [38]:
# # Load the saved model
# model = keras.models.load_model('my_model.keras')
history = model.fit(
    train_ds,
    epochs=1000, 
    batch_size=batch_size,
    verbose=1,
    validation_data=val_ds,
    callbacks=[early_stopping,
              ReduceLROnPlateau,
              ModelCheckpoint]
)

Epoch 1/1000


2025-11-25 05:04:00.234158: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2025-11-25 05:04:05.924321: I external/local_xla/xla/service/service.cc:168] XLA service 0x7cc35db2d1a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-25 05:04:05.924353: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-11-25 05:04:05.930930: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1764047045.964921    6197 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1030/4432 [=====>........................] - ETA: 4:18 - loss: 0.4487 - binary_accuracy: 0.8276

KeyboardInterrupt: 

In [ ]:
model.save('my_model2.keras')

In [ ]:
pretrained_model = keras.models.load_model('my_model2.keras')

In [ ]:
pretrained_model.evaluate(train_ds)